# Data_download_MILANO_hourly_pollutants

In [3]:
#----------------Utils--------------------------
import pandas as pd
import numpy as np
import requests
pd.set_option('display.max_columns', None)

In [4]:
#Remove Outliers function
#This remove a point if it exceeds +-3 std deviations in a window of "window" observations 
# at column "value_column"
def filter_outliers_by_sensor(input_df, window, sensor_list, value_column='value'):
    filtered_df = pd.DataFrame()
    for sensor in sensor_list:
        df = input_df.copy()
        df = df.loc[df['sensor_id'] == sensor]
        #iterate all the df
        df['mean']= df[value_column].rolling(window, center=True, step=1, min_periods=1).mean()
        df['std'] = df[value_column].rolling(window, center=True, step=1, min_periods=1).std()
        df
        #filter setup
        df = df[(df[value_column] <= df['mean']+3*df['std']) & (df[value_column] >= df['mean']-3*df['std'])]
        
        filtered_df = pd.concat([filtered_df, df])
    
    filtered_df = filtered_df.drop(["mean", "std"], axis=1)
    return filtered_df

In [74]:
#global variables
#polution_variables = ['pm10', 'pm25', 'o3', 'no2', 'so2']
polution_variables = ['o3']

pollution_datasets = {
    "pm10": {"value_column": "value", "csv_name": "pm10", "sensor_type": "PM10%20(SM2005)"},
    "pm25": {"value_column": "value", "csv_name": "pm25", "sensor_type": "Particelle%20sospese%20PM2.5"},
    "o3": {"value_column": "value", "csv_name": "o3", "sensor_type": "Ozono"},
    "no2": {"value_column": "value", "csv_name": "no2", "sensor_type": "Biossido%20di%20Azoto"},
    "so2": {"value_column": "value", "csv_name": "so2", "sensor_type": "Biossido%20di%20Zolfo"}
}

date_format = "%Y-%m-%dT%H:%M:%S"

In [6]:
'''
import json
URL = 'http://localhost:8000/api/processes/aggregation-air-quality/execution'
data = {'inputs': {
    "aggregation-time-unit": "year",
    "aggregation-function": "AVG",
    "datetime": "2010-01-01/2023-12-31",
    "pollutant-list": ["PM10 (SM2005)"],
    "sensor-list": [6905, 6907, 6908, 6909, 6912, 6956, 9963, 10273, 10320, 10352, 10354, 20035, 20428, 20429, 20430],
    #"bbox": [8.7059077408448857, 45.1614273647876274, 9.5514392287225149, 45.6421695649354575],
    "skip-geometry": True
}}
# Convert data to JSON format
json_data = json.dumps(data)

# Set the Content-Type header to application/json
headers = {'Content-Type': 'application/json'}
response = requests.post(URL, data=json_data, headers=headers)
'''

'\nimport json\nURL = \'http://localhost:8000/api/processes/aggregation-air-quality/execution\'\ndata = {\'inputs\': {\n    "aggregation-time-unit": "year",\n    "aggregation-function": "AVG",\n    "datetime": "2010-01-01/2023-12-31",\n    "pollutant-list": ["PM10 (SM2005)"],\n    "sensor-list": [6905, 6907, 6908, 6909, 6912, 6956, 9963, 10273, 10320, 10352, 10354, 20035, 20428, 20429, 20430],\n    #"bbox": [8.7059077408448857, 45.1614273647876274, 9.5514392287225149, 45.6421695649354575],\n    "skip-geometry": True\n}}\n# Convert data to JSON format\njson_data = json.dumps(data)\n\n# Set the Content-Type header to application/json\nheaders = {\'Content-Type\': \'application/json\'}\nresponse = requests.post(URL, data=json_data, headers=headers)\n'

In [75]:
#DONT RUN IF NOT NECESSARY. IT TAKES TIME TO GET THE ENTIRE DATASET
#MILANO
import datetime
headers = {'Accept': 'application/json'}
milano_data_basepath = "http://localhost:8000/api/collections/airqualitymeasurement/items?"
milano_sensor_base_path = "http://localhost:8000/api/collections/airqualitysensor/items?"

for variable in polution_variables:
    pollution_dataset = pollution_datasets[variable]
    sensor_type = pollution_dataset['sensor_type']
    sensor_path = f'{milano_sensor_base_path}sensor_type={sensor_type}&province=MI'
    #sensor_path = f'{milano_sensor_base_path}nometiposensore=eq.{sensor_type}'
    print(sensor_path)
    
    sensor_req = requests.get(sensor_path, headers=headers)
    sensors_list = sensor_req.json()
    print(sensors_list)
    variable_stations = []
    for sensor in sensors_list:
        print(sensor['date_stop'])
        if sensor['sensor_id'] in [20035, 20430, 20530, 5624, 5640, 5650, 20574]:
            continue
        if sensor['date_stop'] is None:
            variable_stations.append(sensor['sensor_id'])
        else:
            date_stop = datetime.datetime.strptime(sensor['date_stop'], '%Y-%m-%d')
            start_date = datetime.datetime(2010,1,1)
            if date_stop > start_date:
                variable_stations.append(sensor['sensor_id'])
    #variable_stations = list(map(lambda a: a['sensor_id'], sensors_list))
    variable_paths = []
    for station in variable_stations:
        variable_paths.append(f"{milano_data_basepath}sensor_id={station}&datetime=2010-01-01/2023-12-31&f=geojson&limit=100000")
    
    '''
    variable_stations = list(map(lambda a: a['idsensore'], sensors_list))
    variable_paths = []
    for station in variable_stations:
        variable_paths.append(f"{milano_data_basepath}sensor_id={station}")
    '''
    pollution_datasets[variable]['paths'] = variable_paths.copy()
    pollution_datasets[variable]['sensors'] = variable_stations.copy()


http://localhost:8000/api/collections/airqualitysensor/items?sensor_type=Ozono&province=MI
[{'sensor_type': 'Ozono', 'measurement_unit': 'µg/m³', 'station_id': 551, 'station_name': 'Arconate v. De Gasperi', 'altitude': 182, 'province': 'MI', 'comune': 'Arconate', 'is_historical': False, 'date_start': '1999-01-20', 'date_stop': None, 'utm_north': '5043898.0000', 'utm_east': '488082.0000', 'latitude': '45.5485', 'longitude': '8.8473', 'id': 5707, 'sensor_id': 5707}, {'sensor_type': 'Ozono', 'measurement_unit': 'µg/m³', 'station_id': 536, 'station_name': 'Arese v. Matteotti', 'altitude': 157, 'province': 'MI', 'comune': 'Arese', 'is_historical': True, 'date_start': '1995-10-03', 'date_stop': '2018-01-01', 'utm_north': '5043325.0000', 'utm_east': '506302.0000', 'latitude': '45.5434', 'longitude': '9.0807', 'id': 5708, 'sensor_id': 5708}, {'sensor_type': 'Ozono', 'measurement_unit': 'µg/m³', 'station_id': 544, 'station_name': 'Cormano v. Edison', 'altitude': 153, 'province': 'MI', 'comune':

In [76]:
print(len(pollution_datasets['o3']['paths']))
pollution_datasets['o3']['paths']

16


['http://localhost:8000/api/collections/airqualitymeasurement/items?sensor_id=5707&datetime=2010-01-01/2023-12-31&f=geojson&limit=100000',
 'http://localhost:8000/api/collections/airqualitymeasurement/items?sensor_id=5708&datetime=2010-01-01/2023-12-31&f=geojson&limit=100000',
 'http://localhost:8000/api/collections/airqualitymeasurement/items?sensor_id=5710&datetime=2010-01-01/2023-12-31&f=geojson&limit=100000',
 'http://localhost:8000/api/collections/airqualitymeasurement/items?sensor_id=5711&datetime=2010-01-01/2023-12-31&f=geojson&limit=100000',
 'http://localhost:8000/api/collections/airqualitymeasurement/items?sensor_id=5712&datetime=2010-01-01/2023-12-31&f=geojson&limit=100000',
 'http://localhost:8000/api/collections/airqualitymeasurement/items?sensor_id=5714&datetime=2010-01-01/2023-12-31&f=geojson&limit=100000',
 'http://localhost:8000/api/collections/airqualitymeasurement/items?sensor_id=5715&datetime=2010-01-01/2023-12-31&f=geojson&limit=100000',
 'http://localhost:8000/api

In [20]:
'''
url = 'http://localhost:8000/api/collections/airqualitymeasurement/items?sensor_id=6908&datetime=2022-01-01/2023-12-31&f=geojson&limit=10'
req = requests.get(url, headers=headers)
req.json()['features']
'''

"\nurl = 'http://localhost:8000/api/collections/airqualitymeasurement/items?sensor_id=6908&datetime=2022-01-01/2023-12-31&f=geojson&limit=10'\nreq = requests.get(url, headers=headers)\nreq.json()['features']\n"

In [77]:
#request data from the HARMONIA API
headers = {'Accept': 'application/json'}
'''
{'type': 'Feature',
  'id': 9025762,
  'properties': {'date': '2022-01-01T00:00:00',
   'value': '90.0000',
   'sensor_id': 6908},
  'geometry': {'type': 'Point',
   'coordinates': [9.327360033215143, 45.48363035323249]}}
'''

for variable in polution_variables:
    print(f"fetching data for {variable}")
    variable_data = []

    for variable_path in pollution_datasets[variable]['paths']:
        print()
        print(f'fetching {variable_path}')
        r = requests.get(variable_path, headers=headers)
        req_data = r.json()

        next_page = True
        
        while next_page:
            req_features = req_data["features"]
            if len(req_features) > 0:
                req_list = list(map(lambda element: {
                    #"id": element["id"],
                    "date": element["properties"]["date"],
                    "sensor_id": element["properties"]["sensor_id"],
                    "value": float(element["properties"]["value"]),
                    "lat": element["geometry"]["coordinates"][1],
                    "lng": element["geometry"]["coordinates"][0]
                }, req_features))
                variable_data += req_list
            
            req_links = req_data["links"]
            next_page = False
            for l in req_links:
                if l["rel"] == "next":
                    next_link = l["href"]
                    print(f'fetching next page {next_link}')
                    r = requests.get(next_link, headers=headers)
                    req_data = r.json()
                    next_page = True
        
    print(f"END fetching data for {variable}")
    print(f"Building dataframe for {variable}")
    df = pd.DataFrame(variable_data)
    pollution_datasets[variable]['raw'] = df.copy()
    #df = df.drop(["utm_nord", "utm_est"], axis=1)
    df = df.replace(-9999.0, np.nan).dropna(subset=['value'])
    df['date'] = pd.to_datetime(df['date'],  format=date_format)
    df = df.sort_values(by='date')
    window = 30 #days
    df = filter_outliers_by_sensor(
        df, 
        window,
        pollution_datasets[variable]['sensors'],
        'value'
    )
    df = df.sort_values(by='date')
    pollution_datasets[variable]['df'] = df.copy()
    print(f"END Building dataframe for {variable}")
    
    

fetching data for o3

fetching http://localhost:8000/api/collections/airqualitymeasurement/items?sensor_id=5707&datetime=2010-01-01/2023-12-31&f=geojson&limit=100000
fetching next page http://10.78.3.177:8000/api/collections/airqualitymeasurement/items?sensor_id=5707&datetime=2010-01-01/2023-12-31&f=geojson&limit=100000&offset=100000

fetching http://localhost:8000/api/collections/airqualitymeasurement/items?sensor_id=5708&datetime=2010-01-01/2023-12-31&f=geojson&limit=100000

fetching http://localhost:8000/api/collections/airqualitymeasurement/items?sensor_id=5710&datetime=2010-01-01/2023-12-31&f=geojson&limit=100000
fetching next page http://10.78.3.177:8000/api/collections/airqualitymeasurement/items?sensor_id=5710&datetime=2010-01-01/2023-12-31&f=geojson&limit=100000&offset=100000

fetching http://localhost:8000/api/collections/airqualitymeasurement/items?sensor_id=5711&datetime=2010-01-01/2023-12-31&f=geojson&limit=100000

fetching http://localhost:8000/api/collections/airqualitym

In [78]:
#pollution_datasets['pm10']['raw'] = pollution_datasets['pm10']['raw'].drop(['id'])
pollution_datasets['o3']['df']

,date,sensor_id,value,lat,lng
51074,2010-01-01 01:00:00,5707,2.9,45.548517,8.847322
360556,2010-01-01 01:00:00,5712,3.6,45.539769,9.486885
594054,2010-01-01 01:00:00,5717,4.9,45.483630,9.327360
1267986,2010-01-01 01:00:00,10282,5.7,45.478996,9.235491
420598,2010-01-01 01:00:00,5714,12.9,45.324517,9.134517
...,...,...,...,...,...
232276,2023-12-31 00:00:00,5710,2.6,45.551042,9.162614
1267985,2023-12-31 00:00:00,10282,3.9,45.478996,9.235491
1079331,2023-12-31 00:00:00,5725,1.7,45.463347,9.195325
51073,2023-12-31 00:00:00,5707,2.1,45.548517,8.847322


In [79]:
len(pollution_datasets['o3']['df']['sensor_id'].unique())

16

In [24]:
def import_df(path, start_year, end_year, date_format="%Y-%m-%d", date_column="date"):
    df = pd.read_csv(path, index_col = 0)
    if date_format is not None and date_column is not None:
        df[date_column] = pd.to_datetime(df[date_column],  format=date_format)
    df = df.loc[
        (df['date'].dt.year >= start_year) &
        (df['date'].dt.year <= end_year)
    ]
    return df

In [80]:
csv_base_path = '../data/mcm_pollutant_data'
pollution_datasets['o3']['df'].to_csv(f'{csv_base_path}/o3_hourly.csv')

In [74]:
csv_base_path = '../data/milano_pollutant_data'
for pollutant in polution_variables:
    if pollutant in ['pm10','pm25']:
        legacy = import_df(f'{csv_base_path}/{pollutant}_daily.csv',1998,2021)
        new = pd.concat([legacy, pollution_datasets[pollutant]['df']])
        new.to_csv(f'{csv_base_path}/{pollutant}_daily.csv')
    else:
        legacy = import_df(f'{csv_base_path}/{pollutant}_hourly.csv',1998,2021)
        new = pd.concat([legacy, pollution_datasets[pollutant]['df']])
        new.to_csv(f'{csv_base_path}/{pollutant}_hourly.csv')

/tmp/ipykernel_548018/2656353458.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, index_col = 0)
/tmp/ipykernel_548018/2656353458.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, index_col = 0)
/tmp/ipykernel_548018/2656353458.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, index_col = 0)
